# Homework 19: Blockchain with Python

## Imports

In [61]:
# Import dependencies
import subprocess
import json
import os
import pprint
from dotenv import load_dotenv

# Load and set environment variables
load_dotenv('token.env')
mnemonic = os.getenv('mnemonic')

# Import constants.py and necessary functions from bit and web3
from constants import *
from bit import wif_to_key
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
from getpass import getpass

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

### Check constants and mnemonic

In [14]:
# # Check imported constants and mnemonic
# print(ETH, BTCTEST, BTC)
# print(mnemonic)

### Chatter function

In [15]:
# "Chatter" function from Project 1 that prints lines one character at a time
import time
import sys
from random import randrange

# def chatter(npc_chatter):
#     print()
#     for c in npc_chatter:
#         sys.stdout.write(c)
#         sys.stdout.flush()
#         secs = '0.02'# + str(randrange(10,20,1))
#         secs = float(secs)
#         time.sleep(secs)
#     print()
#     time.sleep(1)

# Import function from 'chatter.py'
from chatter import chatter

chatter('Testing chatter function..')
chatter('.........')
chatter('Testing complete.')
chatter('------------')

# Shortened function for testing
# def chatter(npc_chatter):
#     print(npc_chatter)
# chatter('Test test')

Testing chatter function..
.........
Testing complete.
------------


In [16]:
# Toggle chatter
chatter('Would you like to disable chatter function for faster text?')
chatter_io = input('Disable chatter function? (y/n): ')
if chatter_io.lower() == 'y':
    def chatter(npc_chatter):
        print(npc_chatter)
    chatter('Chatter is disabled.')
else:
    from chatter import chatter
    chatter('Chatter is enabled.')
    pass

Would you like to disable chatter function for faster text?
Chatter is disabled.


In [17]:
chatter('Test test test test test')

Test test test test test


## Derive Wallet Keys

### Tests

In [18]:
# # Test `./derive` script from README
# command = 'php ./derive --key=xprv9zbB6Xchu2zRkf6jSEnH9vuy7tpBuq2njDRr9efSGBXSYr1QtN8QHRur28QLQvKRqFThCxopdS1UD61a5q6jGyuJPGLDV9XfYHQto72DAE8 --cols=path,address --coin=ZEC --numderive=3 -g --format=json'

# p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
# output, err = p.communicate()
# p_status = p.wait()

# keys = json.loads(output)
# keys

#### BTC-test derive_wallet function

In [19]:
# test = "BTC-test"

In [20]:
# # Create a function called `derive_wallets`
# # Derive 'btc-test' wallet keys
# # '--coin' flag does not accept the BTCTEST constant when passed in the function by itself
# def derive_btc(command, stdout=subprocess.PIPE, shell=True):
#     command = 'php ./derive -g --mnemonic=mnemonic --coin="btc-test" --numderive=3 --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json'
#     p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     output, err = p.communicate()
#     p_status = p.wait()
#     return json.loads(output)

# derive_btc(command)

#### ETH derive_wallet function

In [21]:
# # Derive 'eth' wallet keys
# def derive_eth(command, stdout=subprocess.PIPE, shell=True):
#     command = 'php ./derive -g --mnemonic=mnemonic --coin=ETH --numderive=3 --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json'
#     p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     output, err = p.communicate()
#     p_status = p.wait()
#     return json.loads(output)

# derive_eth(command)

### `derive_wallets` function

In [22]:
# Derive both sets of wallet keys in one function
def derive_wallets(coin, stdout=subprocess.PIPE, shell=True):
    chatter(f'Enter mnemonic for {coin}. Leave blank to use test wallet.')
    user = input('Enter mnemonic without quotes: ')
    if user == '':
        chatter(f'No mnemonic specified. Accessing test {coin} wallet..')
        phrase = f'"{mnemonic}"'
        # print(phrase)
    else:
        chatter(f'Checking for user\'s {coin} wallet..')
        phrase = f'"{user}"'
        # print(phrase)
    
    command = f'php ./derive -g --mnemonic={phrase} --coin={coin} --numderive=2 --cols=address,index,path,privkey,pubkey --format=json'    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    # wallet = return json.loads(output)
    # return wallet
    
    # Create global variables for the wallets
    if coin == ETH:
        global eth_wallet
        eth_wallet = json.loads(output)
        return json.loads(output)
    elif coin == BTCTEST:
        global btctest_wallet
        btctest_wallet = json.loads(output)
        return json.loads(output)

In [23]:
# # Test function and wallet variables
# derive_wallets(ETH, 2)
# derive_wallets(BTCTEST, 2)

# eth_wallet
# btctest_wallet

### `coins` dict

In [24]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(ETH),
    BTCTEST: derive_wallets(BTCTEST),
    }

coins

Enter mnemonic for eth. Leave blank to use test wallet.
No mnemonic specified. Accessing test eth wallet..
Enter mnemonic for btc-test. Leave blank to use test wallet.
No mnemonic specified. Accessing test btc-test wallet..


{'eth': [{'address': '0x9528797349d45DBa807F63f7b57373d53f5EAD64',
   'index': 0,
   'path': "m/44'/60'/0'/0/0",
   'privkey': '0x2c79af3249bc7893b4af36f7b65f38147c87a422635e240cb723d59411ef2472',
   'pubkey': '03602fbdbca80f97f4ab6aacb504aea9a7cd435cf39395f52ff72baca1c5c6a966'},
  {'address': '0x26077A0eaA11E133739F070b001D28267264D24A',
   'index': 1,
   'path': "m/44'/60'/0'/0/1",
   'privkey': '0xb39311ad2712a9ebce73b92bdad2e04ae859321baf27e845411c83b446c5eb56',
   'pubkey': '0293869e8e611afb86bb8a65cafbf3c37dcbf9f797a8ef49eb86fdb6e2bfa2dcf9'}],
 'btc-test': [{'address': 'mwY8rKZDHjzxx9JYCkfBWXfC89n7LkeRm3',
   'index': 0,
   'path': "m/44'/1'/0'/0/0",
   'privkey': 'cSPGhcfZUpNwJuyBgHB1qA2QMeWXLYrZSrGKhppaEkDpejkhzqcj',
   'pubkey': '036c02e87ffb626f71c32c2433aa14e5be517cb16e79db055116fbcb063c7a17a5'},
  {'address': 'mnLfB25RMps3ABSqK6orbwQDQdMhC1FvS2',
   'index': 1,
   'path': "m/44'/1'/0'/0/1",
   'privkey': 'cUrNRHgxRmpdxzxcCXi1YgNwpyDtmboT5fAi989kyg6uJKbFy34r',
   'pubkey': '

In [25]:
# # Check access to `coins` dict
# print(f"ETH private key: {coins[ETH][0]['privkey']}")
# print(f"BTCTEST private key: {coins[BTCTEST][0]['privkey']}")

## Linking Transaction Signing Libraries

### `priv_key_to_account` function

In [26]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        # Create global variables for account data
        # global eth_account
        # eth_account = Account.privateKeyToAccount(priv_key)
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        # global btctest_account
        # btctest_account = PrivateKeyTestnet(priv_key)
        return PrivateKeyTestnet(priv_key)

In [27]:
# # Test function and global variables
# print(priv_key_to_account(ETH, eth_privkey))
# print(eth_account)
# print()
# print(priv_key_to_account(BTCTEST, btctest_privkey))
# print(btctest_account)

In [88]:
print(f'ETH balance (in wei): {w3.eth.get_balance(eth_address)}')
print(f'ETH balance (in ETH): {w3.fromWei(w3.eth.get_balance(eth_address), "ether")}')

ETH balance (in wei): 94999411999999999930
ETH balance (in ETH): 94.99941199999999993
ETH balance (in wei): 94999411999999999930


#### Tests

In [28]:
# # Tests for syntax and object types
# eprivkey = Account.privateKeyToAccount(coins[ETH][0]['privkey'])
# print(f'{eprivkey} || {type(eprivkey)}\n')

# bprivkey = coins[BTCTEST][0]['privkey']
# bwifkey = wif_to_key(bprivkey)
# print(f'{bwifkey} || {type(bwifkey)}')
# print(f'{PrivateKeyTestnet(bprivkey)} || {type(bwifkey)}\n')

# print(priv_key_to_account(ETH, 0))
# print(priv_key_to_account(BTCTEST, 0))

In [29]:
# # Test bit functions
# key = priv_key_to_account(BTCTEST, 0)

# print(f'# BTCTEST Balance: {key.get_balance("btc")}\n')

# print(f'# BTCTEST Balance in USD: ${key.balance_as("usd")}\n')

# print(f'# BTCTEST Transactions:\n{key.get_transactions()[:3]}\n')

# print(f'# BTCTEST Unspent:\n{key.get_unspents()}\n')

# # PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [30]:
# # Test web3 functions
# w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
# w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# def create_raw_tx(account, recipient, amount):
#     gasEstimate = w3.eth.estimateGas(
#         {"from": account.address, "to": recipient, "value": amount}
#     )
#     return {
#         "from": account.address,
#         "to": recipient,
#         "value": amount,
#         "gasPrice": w3.eth.gasPrice,
#         "gas": gasEstimate,
#         "nonce": w3.eth.getTransactionCount(account.address),
#     }


### `create_tx` function

In [70]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    chatter(f'Creating unsigned {coin} transaction..')
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {'from': account.address, 'to': to, 'value': amount}
        )
        return {
            'from': account.address,
            'to': to,
            'value': amount,
            'gasPrice': w3.eth.gasPrice,
            'gas': gasEstimate,
            'nonce': w3.eth.getTransactionCount(account.address),
            # 'chainID': w3.eth.chain_id
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [32]:
# Test function
# create_tx(BTCTEST, priv_key_to_account(BTCTEST, derive_wallets(BTCTEST)), 'mnLfB25RMps3ABSqK6orbwQDQdMhC1FvS2', .00001)
# create_tx(ETH, priv_key_to_account(ETH, derive_wallets(ETH)), '0x6810BE478c13290ED8e9dda715e3Beb6D401Ac9C', .00001)

### `send_tx` function

In [77]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account, to, amount)
        chatter('Unsigned transaction created.')
        chatter('Sending transaction..')
        signed_tx = account.sign_transaction(raw_tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        chatter('Transaction sent.')
        return result.hex()
    elif coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        chatter('Unsigned transaction created.')
        chatter('Sending transaction..')
        signed_tx = account.sign_transaction(raw_tx)
        chatter('Transaction sent.')
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

### Wallet variables

In [84]:
# Create wallet variables
eth_account = priv_key_to_account(ETH, eth_privkey)
eth_address = coins[ETH][0]['address']
eth_privkey = coins[ETH][0]['privkey']
eth_receiver = '0x26077A0eaA11E133739F070b001D28267264D24A'     # second eth wallet
btctest_account = priv_key_to_account(BTCTEST, btctest_privkey)
btctest_address = coins[BTCTEST][0]['address']
btctest_privkey = coins[BTCTEST][0]['privkey']
btctest_receiver = 'mnLfB25RMps3ABSqK6orbwQDQdMhC1FvS2'     # second btctest wallet

# Check values
# print(f'ETH address: {eth_address}\nprivkey: {eth_privkey}')
# print()
# print(f'BTCTEST address: {btctest_address}\nprivkey: {btctest_privkey}')

# print(f'ETH privkey: {json.dumps(eth_privkey)}')
# print(f'BTCTEST privkey: {json.dumps(btctest_privkey)}')

---

## Send some transactions

Generate wallet based on 'coin' input  
`derive_wallets(coin)`

Runs derive_wallets(coin) to generate both wallets', and to store addresses and priv_keys  
`coins`

Generate accounts based on parameters 'coin' and 'priv_key'  
`priv_key_to_account(coin, priv_key)`

Creates transaction based on parameters 'coin', 'account', 'to', 'amount'  
`create_tx(coin, account, to, amount)`

Runs create_tx() with associated parameters, then sends transaction  
`send_tx(coin, account, to, amount)`


In [35]:
coins

{'eth': [{'address': '0x9528797349d45DBa807F63f7b57373d53f5EAD64',
   'index': 0,
   'path': "m/44'/60'/0'/0/0",
   'privkey': '0x2c79af3249bc7893b4af36f7b65f38147c87a422635e240cb723d59411ef2472',
   'pubkey': '03602fbdbca80f97f4ab6aacb504aea9a7cd435cf39395f52ff72baca1c5c6a966'},
  {'address': '0x26077A0eaA11E133739F070b001D28267264D24A',
   'index': 1,
   'path': "m/44'/60'/0'/0/1",
   'privkey': '0xb39311ad2712a9ebce73b92bdad2e04ae859321baf27e845411c83b446c5eb56',
   'pubkey': '0293869e8e611afb86bb8a65cafbf3c37dcbf9f797a8ef49eb86fdb6e2bfa2dcf9'}],
 'btc-test': [{'address': 'mwY8rKZDHjzxx9JYCkfBWXfC89n7LkeRm3',
   'index': 0,
   'path': "m/44'/1'/0'/0/0",
   'privkey': 'cSPGhcfZUpNwJuyBgHB1qA2QMeWXLYrZSrGKhppaEkDpejkhzqcj',
   'pubkey': '036c02e87ffb626f71c32c2433aa14e5be517cb16e79db055116fbcb063c7a17a5'},
  {'address': 'mnLfB25RMps3ABSqK6orbwQDQdMhC1FvS2',
   'index': 1,
   'path': "m/44'/1'/0'/0/1",
   'privkey': 'cUrNRHgxRmpdxzxcCXi1YgNwpyDtmboT5fAi989kyg6uJKbFy34r',
   'pubkey': '

### ETH Transaction

In [37]:
priv_key_to_account(ETH, eth_privkey)

In [72]:
# send_tx(ETH, eth_account, eth_receiver, 5)

Creating unsigned eth transaction..
Unsigned transaction created.
Sending transaction..
Transaction sent.
0x949ab560eaa1fcdaf3602a993b7c9cece329c62f1a5a423f8fb0bff0bcd5dbcb


'0x949ab560eaa1fcdaf3602a993b7c9cece329c62f1a5a423f8fb0bff0bcd5dbcb'

In [79]:
# Check balance and number of transactions
print(w3.eth.get_balance(coins[ETH][0]['address']))
print(w3.eth.get_transaction_count(coins[ETH][0]['address']))

94999411999999999930
5


In [74]:
# Check transaction using hash
print(w3.eth.get_transaction('0x949ab560eaa1fcdaf3602a993b7c9cece329c62f1a5a423f8fb0bff0bcd5dbcb'))

AttributeDict({'hash': HexBytes('0x949ab560eaa1fcdaf3602a993b7c9cece329c62f1a5a423f8fb0bff0bcd5dbcb'), 'nonce': 2, 'blockHash': HexBytes('0x20404e95c1c6774a8d5a48f95e4ecea6d43709409d79c22331058b0713397eb2'), 'blockNumber': 3, 'transactionIndex': 0, 'from': '0x9528797349d45DBa807F63f7b57373d53f5EAD64', 'to': '0x26077A0eaA11E133739F070b001D28267264D24A', 'value': 5, 'gas': 21000, 'gasPrice': 2000000000, 'input': '0x', 'v': 27, 'r': HexBytes('0xee67dc821afbe0702b1c2688e3b5600785ca27264addb0e9d123977f9b8abeff'), 's': HexBytes('0x3a549a598c68f8fcd74cd68058d874b7b311c5bcecef8eb369b029e0f8908a16')})


### BTCTEST Transaction

In [75]:
priv_key_to_account(BTCTEST, btctest_privkey)

<PrivateKeyTestnet: mwY8rKZDHjzxx9JYCkfBWXfC89n7LkeRm3>

In [78]:
send_tx(BTCTEST, btctest_account, btctest_receiver, 0.00001)

Creating unsigned btc-test transaction..
Unsigned transaction created.
Sending transaction..
Transaction sent.
None


In [43]:
# Get transaction history
btctest_account.get_transactions()

['fc2cf8573ef0eb2f9428c580d3ddabeeafd657cc40c28386eb128cd53117b497',
 'bbeb2fd9c9008df58ce5387a257df8cac49b37e0abb3010fcbbe673d1bada9ee',
 'e9e6b0c7ff5772e9e49d30fbb0ac7951be39612e38823e5f3faeb20c58a9f8c3',
 'da2994a751de4c252eb4e3c27304d36e8cfeb01c48b7cfe6ca2b325eecfb0b3a',
 '04cdae499b1a375c0ac1220dadbaa1890e3f379023f72fdf8bc5ea641c27efef',
 '4ed6bbbf59b3b59628593f9e59c0d37d3a82bd12a8d8686e4580e73c22bf12b5',
 'dc65c9859abea9eae7584eba7aa82f7974c405fba7b19740fc275d40de6bf7ad',
 'f362f86360812f1b38e9e860796947340b07d2245256de15e8799a31a8601259',
 '2b1b2a689c72b98fe82578c0c0dcf59a5d027335e9b8efa590632502fde75827',
 '3d29856649840461de837e1d0fe507d116a2619fd5fcad887f80002c108c8ca2',
 '0539093c1fb792560616ee2d8f0e6e9c1b2ee62d7edd9f19ed8d2c372f8b6ef1',
 '7e00ac895062470221f7df662f973a0715ebfabb0d24446d6fec95dc5fddb4b2']

In [44]:
# Get balance
btctest_account.get_balance('btc')

'0.00221156'